# পুনরাবৃত্তি নিউরাল নেটওয়ার্ক

পূর্ববর্তী মডিউলে, আমরা টেক্সটের সমৃদ্ধ অর্থবহ উপস্থাপনাগুলি নিয়ে আলোচনা করেছি। আমরা যে আর্কিটেকচারটি ব্যবহার করেছি তা বাক্যের শব্দগুলির সম্মিলিত অর্থ ধারণ করে, তবে এটি শব্দগুলির **ক্রম** বিবেচনা করে না, কারণ এম্বেডিংয়ের পরে যে সংযোজন প্রক্রিয়া হয় তা মূল টেক্সট থেকে এই তথ্যটি সরিয়ে দেয়। যেহেতু এই মডেলগুলি শব্দের ক্রম উপস্থাপন করতে অক্ষম, তাই তারা টেক্সট জেনারেশন বা প্রশ্নোত্তরের মতো আরও জটিল বা দ্ব্যর্থক কাজ সমাধান করতে পারে না।

একটি টেক্সট সিকোয়েন্সের অর্থ ধরতে, আমরা **পুনরাবৃত্তি নিউরাল নেটওয়ার্ক** বা RNN নামে একটি নিউরাল নেটওয়ার্ক আর্কিটেকচার ব্যবহার করব। RNN ব্যবহার করার সময়, আমরা আমাদের বাক্যটি নেটওয়ার্কের মধ্য দিয়ে একবারে একটি টোকেন পাঠাই, এবং নেটওয়ার্কটি কিছু **অবস্থা** তৈরি করে, যা আমরা পরবর্তী টোকেনের সাথে আবার নেটওয়ার্কে পাঠাই।

![একটি উদাহরণ পুনরাবৃত্তি নিউরাল নেটওয়ার্ক জেনারেশন দেখানো চিত্র।](../../../../../translated_images/bn/rnn.27f5c29c53d727b5.webp)

টোকেনগুলির ইনপুট সিকোয়েন্স $X_0,\dots,X_n$ দেওয়া হলে, RNN একটি নিউরাল নেটওয়ার্ক ব্লকের সিকোয়েন্স তৈরি করে এবং ব্যাকপ্রোপাগেশনের মাধ্যমে এই সিকোয়েন্সটি এন্ড-টু-এন্ড প্রশিক্ষণ দেয়। প্রতিটি নেটওয়ার্ক ব্লক $(X_i,S_i)$ জোড়াকে ইনপুট হিসাবে নেয় এবং ফলাফল হিসাবে $S_{i+1}$ তৈরি করে। চূড়ান্ত অবস্থা $S_n$ বা আউটপুট $Y_n$ একটি লিনিয়ার ক্লাসিফায়ারে যায় ফলাফল তৈরি করতে। সমস্ত নেটওয়ার্ক ব্লক একই ওজন ভাগ করে এবং একটি ব্যাকপ্রোপাগেশন পাস ব্যবহার করে এন্ড-টু-এন্ড প্রশিক্ষণ দেওয়া হয়।

> উপরের চিত্রটি পুনরাবৃত্তি নিউরাল নেটওয়ার্ককে আনরোলড ফর্মে (বামে) এবং আরও কমপ্যাক্ট পুনরাবৃত্তি উপস্থাপনায় (ডানে) দেখায়। এটি বোঝা গুরুত্বপূর্ণ যে সমস্ত RNN সেলগুলির একই **শেয়ারযোগ্য ওজন** রয়েছে।

কারণ অবস্থা ভেক্টর $S_0,\dots,S_n$ নেটওয়ার্কের মধ্য দিয়ে যায়, RNN শব্দগুলির মধ্যে ক্রমিক নির্ভরতা শিখতে সক্ষম। উদাহরণস্বরূপ, যখন *not* শব্দটি সিকোয়েন্সের কোথাও উপস্থিত হয়, এটি অবস্থা ভেক্টরের নির্দিষ্ট উপাদানগুলিকে অস্বীকার করতে শিখতে পারে।

প্রত্যেকটি RNN সেলের ভিতরে দুটি ওজন ম্যাট্রিক্স থাকে: $W_H$ এবং $W_I$, এবং বায়াস $b$। প্রতিটি RNN ধাপে, ইনপুট $X_i$ এবং ইনপুট অবস্থা $S_i$ দেওয়া হলে, আউটপুট অবস্থা হিসাব করা হয় $S_{i+1} = f(W_H\times S_i + W_I\times X_i+b)$, যেখানে $f$ একটি অ্যাক্টিভেশন ফাংশন (সাধারণত $\tanh$)।

> টেক্সট জেনারেশন (যা আমরা পরবর্তী ইউনিটে আলোচনা করব) বা মেশিন ট্রান্সলেশনের মতো সমস্যার জন্য আমরা প্রতিটি RNN ধাপে কিছু আউটপুট মানও পেতে চাই। এই ক্ষেত্রে, আরেকটি ম্যাট্রিক্স $W_O$ থাকে, এবং আউটপুট হিসাব করা হয় $Y_i=f(W_O\times S_i+b_O)$।

চলুন দেখি কীভাবে পুনরাবৃত্তি নিউরাল নেটওয়ার্ক আমাদের নিউজ ডেটাসেট শ্রেণীবদ্ধ করতে সাহায্য করতে পারে।

> স্যান্ডবক্স পরিবেশের জন্য, আমাদের নিশ্চিত করতে হবে যে প্রয়োজনীয় লাইব্রেরি ইনস্টল করা হয়েছে এবং ডেটা প্রিফেচ করা হয়েছে, এজন্য নিম্নলিখিত সেলটি চালাতে হবে। আপনি যদি লোকাল মেশিনে কাজ করেন, তবে এই সেলটি এড়িয়ে যেতে পারেন।


In [1]:
import sys
!{sys.executable} -m pip install --quiet tensorflow_datasets==4.4.0
!cd ~ && wget -q -O - https://mslearntensorflowlp.blob.core.windows.net/data/tfds-ag-news.tgz | tar xz

In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

# We are going to be training pretty large models. In order not to face errors, we need
# to set tensorflow option to grow GPU memory allocation when required
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

ds_train, ds_test = tfds.load('ag_news_subset').values()

বড় মডেল প্রশিক্ষণ করার সময় GPU মেমোরি বরাদ্দ একটি সমস্যা হয়ে উঠতে পারে। এছাড়াও, আমাদের বিভিন্ন মিনিব্যাচ সাইজ নিয়ে পরীক্ষা করতে হতে পারে, যাতে ডেটা GPU মেমোরিতে ফিট করে, এবং প্রশিক্ষণ যথেষ্ট দ্রুত হয়। যদি আপনি আপনার নিজস্ব GPU মেশিনে এই কোড চালাচ্ছেন, তাহলে প্রশিক্ষণের গতি বাড়ানোর জন্য মিনিব্যাচ সাইজ সামঞ্জস্য করার চেষ্টা করতে পারেন।

> **Note**: NVidia ড্রাইভারের কিছু নির্দিষ্ট সংস্করণ মডেল প্রশিক্ষণের পর মেমোরি মুক্ত করতে ব্যর্থ হতে পারে। আমরা এই নোটবুকে কয়েকটি উদাহরণ চালাচ্ছি, এবং এটি কিছু সেটআপে মেমোরি শেষ হয়ে যাওয়ার কারণ হতে পারে, বিশেষত যদি আপনি একই নোটবুকের অংশ হিসেবে নিজের পরীক্ষা চালাচ্ছেন। যদি আপনি মডেল প্রশিক্ষণ শুরু করার সময় কিছু অদ্ভুত ত্রুটি সম্মুখীন করেন, তাহলে নোটবুকের কার্নেল পুনরায় চালু করার কথা বিবেচনা করতে পারেন।


In [3]:
batch_size = 16
embed_size = 64

## সহজ RNN শ্রেণীবিভাজক

একটি সহজ RNN-এর ক্ষেত্রে, প্রতিটি পুনরাবৃত্তি ইউনিট একটি সাধারণ রৈখিক নেটওয়ার্ক, যা একটি ইনপুট ভেক্টর এবং স্টেট ভেক্টর গ্রহণ করে এবং একটি নতুন স্টেট ভেক্টর তৈরি করে। Keras-এ, এটি `SimpleRNN` স্তর দ্বারা উপস্থাপন করা যেতে পারে।

যদিও আমরা সরাসরি RNN স্তরে এক-হট এনকোড করা টোকেন পাস করতে পারি, এটি একটি ভালো ধারণা নয় কারণ এগুলোর উচ্চ মাত্রা থাকে। তাই, আমরা শব্দ ভেক্টরের মাত্রা কমানোর জন্য একটি এমবেডিং স্তর ব্যবহার করব, এরপর একটি RNN স্তর এবং শেষে একটি `Dense` শ্রেণীবিভাজক।

> **Note**: এমন ক্ষেত্রে যেখানে মাত্রা এত বেশি নয়, যেমন চরিত্র-স্তরের টোকেনাইজেশন ব্যবহার করার সময়, এক-হট এনকোড করা টোকেন সরাসরি RNN সেলে পাস করা যুক্তিযুক্ত হতে পারে।


In [4]:
vocab_size = 20000

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 64)          1280000   
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 16)                1296      
_________________________________________________________________
dense (Dense)                (None, 4)                 68        
Total params: 1,281,364
Trainable params: 1,281,364
Non-trainable params: 0
_________________________________________________________________


> **নোট:** এখানে আমরা সরলতার জন্য একটি অনট্রেইন্ড এমবেডিং লেয়ার ব্যবহার করছি, তবে আরও ভালো ফলাফলের জন্য আমরা Word2Vec ব্যবহার করে একটি প্রিট্রেইন্ড এমবেডিং লেয়ার ব্যবহার করতে পারি, যা আগের ইউনিটে বর্ণনা করা হয়েছে। এটি আপনার জন্য একটি ভালো অনুশীলন হবে যদি আপনি এই কোডটি প্রিট্রেইন্ড এমবেডিংয়ের সাথে কাজ করার জন্য মানিয়ে নিতে পারেন।

এখন চলুন আমাদের RNN প্রশিক্ষণ করি। সাধারণভাবে RNN প্রশিক্ষণ করা বেশ কঠিন, কারণ RNN সেলগুলো সিকোয়েন্সের দৈর্ঘ্য বরাবর আনরোল করার পর, ব্যাকপ্রোপাগেশনে জড়িত লেয়ারগুলোর সংখ্যা অনেক বেড়ে যায়। তাই আমাদের একটি ছোট লার্নিং রেট নির্বাচন করতে হবে এবং ভালো ফলাফলের জন্য বড় ডেটাসেটে নেটওয়ার্কটি প্রশিক্ষণ দিতে হবে। এটি করতে বেশ সময় লাগতে পারে, তাই GPU ব্যবহার করাই উত্তম।

গতি বাড়ানোর জন্য, আমরা শুধুমাত্র নিউজ শিরোনামে RNN মডেলটি প্রশিক্ষণ দেব, বর্ণনা অংশটি বাদ দিয়ে। আপনি চাইলে বর্ণনা সহ প্রশিক্ষণ দিয়ে দেখতে পারেন যে মডেলটি প্রশিক্ষণ পেতে পারে কিনা।


In [5]:
def extract_title(x):
    return x['title']

def tupelize_title(x):
    return (extract_title(x),x['label'])

print('Training vectorizer')
vectorizer.adapt(ds_train.take(2000).map(extract_title))

Training vectorizer


In [6]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize_title).batch(batch_size),validation_data=ds_test.map(tupelize_title).batch(batch_size))

7500/7500 [==============================] - 82s 11ms/step - loss: 0.6629 - acc: 0.7623 - val_loss: 0.5559 - val_acc: 0.7995


> **নোট** যে এখানে সঠিকতা সম্ভবত কম হবে, কারণ আমরা শুধুমাত্র সংবাদ শিরোনামের উপর প্রশিক্ষণ দিচ্ছি।


## ভেরিয়েবল সিকোয়েন্স পুনর্বিবেচনা

মনে রাখবেন যে `TextVectorization` লেয়ার একটি মিনিব্যাচে ভেরিয়েবল দৈর্ঘ্যের সিকোয়েন্সগুলোকে স্বয়ংক্রিয়ভাবে প্যাড টোকেন দিয়ে পূরণ করবে। দেখা গেছে, এই টোকেনগুলোও প্রশিক্ষণে অংশ নেয় এবং মডেলের কনভার্জেন্স জটিল করে তুলতে পারে।

প্যাডিংয়ের পরিমাণ কমানোর জন্য আমরা কয়েকটি পদ্ধতি অবলম্বন করতে পারি। এর মধ্যে একটি হলো ডেটাসেটকে সিকোয়েন্সের দৈর্ঘ্য অনুযায়ী পুনর্বিন্যাস করা এবং সব সিকোয়েন্সকে তাদের আকার অনুযায়ী গ্রুপ করা। এটি `tf.data.experimental.bucket_by_sequence_length` ফাংশন ব্যবহার করে করা যেতে পারে (দেখুন [ডকুমেন্টেশন](https://www.tensorflow.org/api_docs/python/tf/data/experimental/bucket_by_sequence_length))।

আরেকটি পদ্ধতি হলো **মাস্কিং** ব্যবহার করা। কেরাসে, কিছু লেয়ার অতিরিক্ত ইনপুট সমর্থন করে যা দেখায় কোন টোকেনগুলো প্রশিক্ষণের সময় বিবেচনায় নেওয়া উচিত। আমাদের মডেলে মাস্কিং অন্তর্ভুক্ত করতে, আমরা হয় একটি আলাদা `Masking` লেয়ার ([ডকস](https://keras.io/api/layers/core_layers/masking/)) যোগ করতে পারি, অথবা আমাদের `Embedding` লেয়ারের `mask_zero=True` প্যারামিটার নির্দিষ্ট করতে পারি।

> **Note**: পুরো ডেটাসেটে একটি এপোক সম্পন্ন করতে এই প্রশিক্ষণে প্রায় ৫ মিনিট সময় লাগবে। ধৈর্য হারালে যে কোনো সময় প্রশিক্ষণ বন্ধ করতে পারেন। এছাড়াও, প্রশিক্ষণের জন্য ব্যবহৃত ডেটার পরিমাণ সীমিত করতে পারেন, `ds_train` এবং `ds_test` ডেটাসেটের পরে `.take(...)` ক্লজ যোগ করে।


In [7]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size,embed_size,mask_zero=True),
    keras.layers.SimpleRNN(16),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

7500/7500 [==============================] - 371s 49ms/step - loss: 0.5401 - acc: 0.8079 - val_loss: 0.3780 - val_acc: 0.8822


এখন আমরা মাস্কিং ব্যবহার করছি, তাই আমরা শিরোনাম এবং বিবরণের পুরো ডেটাসেটে মডেলটি প্রশিক্ষণ দিতে পারি।

> **Note**: আপনি কি লক্ষ্য করেছেন যে আমরা নিউজ শিরোনামের উপর প্রশিক্ষিত ভেক্টরাইজার ব্যবহার করছি, পুরো নিবন্ধের মূল অংশের উপর নয়? এটি সম্ভবত কিছু টোকেন উপেক্ষা করতে পারে, তাই ভেক্টরাইজারটি পুনরায় প্রশিক্ষণ দেওয়া ভালো। তবে, এর প্রভাব খুবই সামান্য হতে পারে, তাই সরলতার জন্য আমরা পূর্বে প্রশিক্ষিত ভেক্টরাইজারটিই ব্যবহার করব।


## LSTM: দীর্ঘমেয়াদী স্মৃতি

RNN-এর প্রধান সমস্যাগুলোর একটি হলো **vanishing gradients**। RNN অনেক দীর্ঘ হতে পারে এবং ব্যাকপ্রোপাগেশনের সময় নেটওয়ার্কের প্রথম স্তর পর্যন্ত গ্রেডিয়েন্টগুলো সঠিকভাবে পৌঁছাতে সমস্যা হতে পারে। যখন এটি ঘটে, তখন নেটওয়ার্ক দূরবর্তী টোকেনগুলোর মধ্যে সম্পর্ক শিখতে পারে না। এই সমস্যাটি এড়ানোর একটি উপায় হলো **স্পষ্ট স্টেট ম্যানেজমেন্ট** প্রবর্তন করা, যা **গেট** ব্যবহার করে করা হয়। গেট প্রবর্তনকারী সবচেয়ে সাধারণ দুটি আর্কিটেকচার হলো **দীর্ঘমেয়াদী স্মৃতি** (LSTM) এবং **গেটেড রিলে ইউনিট** (GRU)। এখানে আমরা LSTM নিয়ে আলোচনা করব।

![দীর্ঘমেয়াদী স্মৃতি সেলের একটি উদাহরণ দেখানো চিত্র](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

একটি LSTM নেটওয়ার্ক RNN-এর মতোই সংগঠিত, তবে এখানে দুটি স্টেট স্তর থেকে স্তরে প্রেরণ করা হয়: প্রকৃত স্টেট $c$, এবং হিডেন ভেক্টর $h$। প্রতিটি ইউনিটে, হিডেন ভেক্টর $h_{t-1}$ ইনপুট $x_t$ এর সাথে মিলিত হয় এবং একসাথে তারা **গেট** এর মাধ্যমে স্টেট $c_t$ এবং আউটপুট $h_{t}$ এর উপর কী ঘটবে তা নিয়ন্ত্রণ করে। প্রতিটি গেটের সিগময়েড অ্যাক্টিভেশন থাকে (আউটপুট $[0,1]$ পরিসরে), যা স্টেট ভেক্টরের সাথে গুণ করলে বিটওয়াইজ মাস্ক হিসেবে ভাবা যেতে পারে। LSTM-এ নিম্নলিখিত গেটগুলো থাকে (উপরের ছবিতে বাম থেকে ডানে):
* **forget gate**, যা নির্ধারণ করে ভেক্টর $c_{t-1}$ এর কোন উপাদানগুলো আমরা ভুলে যাব এবং কোনগুলো প্রেরণ করব।
* **input gate**, যা নির্ধারণ করে ইনপুট ভেক্টর এবং পূর্ববর্তী হিডেন ভেক্টর থেকে কতটুকু তথ্য স্টেট ভেক্টরে অন্তর্ভুক্ত করা হবে।
* **output gate**, যা নতুন স্টেট ভেক্টর নিয়ে সিদ্ধান্ত নেয় এর কোন উপাদানগুলো নতুন হিডেন ভেক্টর $h_t$ উৎপাদনের জন্য ব্যবহার করা হবে।

স্টেট $c$ এর উপাদানগুলোকে এমন ফ্ল্যাগ হিসেবে ভাবা যেতে পারে যা চালু এবং বন্ধ করা যায়। উদাহরণস্বরূপ, যখন আমরা সিকোয়েন্সে *Alice* নামটি পাই, আমরা অনুমান করি এটি একজন নারীর নাম এবং স্টেটে একটি ফ্ল্যাগ উত্থাপন করি যা বলে যে বাক্যে একটি নারীবাচক বিশেষ্য রয়েছে। যখন আমরা আরও *and Tom* শব্দগুলো পাই, তখন আমরা একটি ফ্ল্যাগ উত্থাপন করি যা বলে যে আমাদের কাছে একটি বহুবচন বিশেষ্য রয়েছে। এভাবে স্টেট ম্যানিপুলেট করে আমরা বাক্যের ব্যাকরণগত বৈশিষ্ট্যগুলো ট্র্যাক করতে পারি।

> **Note**: LSTM-এর অভ্যন্তরীণ বিষয়গুলো বোঝার জন্য এটি একটি চমৎকার রিসোর্স: [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) ক্রিস্টোফার ওলাহ দ্বারা।

যদিও একটি LSTM সেলের অভ্যন্তরীণ গঠন জটিল মনে হতে পারে, Keras এই বাস্তবায়নটি `LSTM` লেয়ারের ভিতরে লুকিয়ে রাখে, তাই উপরের উদাহরণে আমাদের যা করতে হবে তা হলো পুনরাবৃত্ত স্তরটি প্রতিস্থাপন করা:


In [8]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, embed_size),
    keras.layers.LSTM(8),
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(8),validation_data=ds_test.map(tupelize).batch(8))

15000/15000 [==============================] - 188s 13ms/step - loss: 0.5692 - acc: 0.7916 - val_loss: 0.3441 - val_acc: 0.8870


**বিঃদ্রঃ** LSTM প্রশিক্ষণ দেওয়া বেশ ধীরগতির, এবং প্রশিক্ষণের শুরুতে আপনি হয়তো সঠিকতার ক্ষেত্রে খুব বেশি উন্নতি দেখতে পাবেন না। ভালো সঠিকতা অর্জনের জন্য আপনাকে কিছু সময় ধরে প্রশিক্ষণ চালিয়ে যেতে হতে পারে।


## দ্বিমুখী এবং বহুস্তর RNN

আমাদের আগের উদাহরণগুলোতে, পুনরাবৃত্ত নেটওয়ার্কগুলো একটি সিকোয়েন্সের শুরু থেকে শেষ পর্যন্ত কাজ করে। এটি আমাদের কাছে স্বাভাবিক মনে হয় কারণ এটি সেই দিক অনুসরণ করে যেভাবে আমরা পড়ি বা কথা শুনি। তবে এমন পরিস্থিতিতে যেখানে ইনপুট সিকোয়েন্সের র‍্যান্ডম অ্যাক্সেস প্রয়োজন, সেখানে পুনরাবৃত্ত গণনা উভয় দিকেই চালানো আরও যুক্তিসঙ্গত। এমন RNN-গুলো, যেগুলো উভয় দিকেই গণনা করতে পারে, তাদের **দ্বিমুখী** RNN বলা হয়, এবং এগুলো তৈরি করা যায় পুনরাবৃত্ত স্তরকে একটি বিশেষ `Bidirectional` স্তর দিয়ে মোড়ানোর মাধ্যমে।

> **Note**: `Bidirectional` স্তরটি তার মধ্যে থাকা স্তরের দুটি কপি তৈরি করে এবং সেই কপিগুলোর একটির `go_backwards` প্রপার্টিকে `True` সেট করে, যা সিকোয়েন্সের বিপরীত দিকে কাজ করে।

পুনরাবৃত্ত নেটওয়ার্ক, একমুখী বা দ্বিমুখী, সিকোয়েন্সের মধ্যে প্যাটার্নগুলো ধরে এবং সেগুলোকে স্টেট ভেক্টরে সংরক্ষণ করে বা আউটপুট হিসেবে ফেরত দেয়। কনভোলিউশনাল নেটওয়ার্কের মতো, আমরা প্রথম স্তরের মাধ্যমে নিম্ন স্তরের প্যাটার্নগুলো থেকে উচ্চ স্তরের প্যাটার্ন ধরার জন্য আরেকটি পুনরাবৃত্ত স্তর তৈরি করতে পারি। এটি আমাদের **বহুস্তর RNN** ধারণার দিকে নিয়ে যায়, যা দুটি বা তার বেশি পুনরাবৃত্ত নেটওয়ার্ক নিয়ে গঠিত, যেখানে আগের স্তরের আউটপুট পরবর্তী স্তরের ইনপুট হিসেবে ব্যবহৃত হয়।

![একটি বহুস্তর দীর্ঘ-মেয়াদী-মেমরি RNN-এর ছবি](../../../../../translated_images/bn/multi-layer-lstm.dd975e29bb2a59fe.webp)

*ছবি [এই চমৎকার পোস্ট](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) থেকে, লেখক Fernando López।*

Keras এই নেটওয়ার্কগুলো তৈরি করাকে সহজ করে তোলে, কারণ আপনাকে শুধু মডেলে আরও পুনরাবৃত্ত স্তর যোগ করতে হবে। শেষ স্তর ছাড়া অন্য সব স্তরের জন্য, আমাদের `return_sequences=True` প্যারামিটার নির্দিষ্ট করতে হবে, কারণ আমাদের স্তর থেকে সমস্ত মধ্যবর্তী স্টেট ফেরত দরকার, শুধুমাত্র পুনরাবৃত্ত গণনার চূড়ান্ত স্টেট নয়।

চলুন আমাদের শ্রেণীবিন্যাস সমস্যার জন্য একটি দুই-স্তর বিশিষ্ট দ্বিমুখী LSTM তৈরি করি।

> **Note** এই কোডটি সম্পন্ন হতে আবারও বেশ সময় নেয়, তবে এটি আমাদের এখন পর্যন্ত দেখা সর্বোচ্চ সঠিকতা প্রদান করে। তাই হয়তো অপেক্ষা করা এবং ফলাফল দেখা মূল্যবান।


In [9]:
model = keras.models.Sequential([
    vectorizer,
    keras.layers.Embedding(vocab_size, 128, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64,return_sequences=True)),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),    
    keras.layers.Dense(4,activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

5045/7500 [===================>..........] - ETA: 2:33 - loss: 0.3709 - acc: 0.8706

## অন্যান্য কাজের জন্য RNNs

এখন পর্যন্ত, আমরা RNNs ব্যবহার করে টেক্সট সিকোয়েন্স শ্রেণীবদ্ধ করার উপর জোর দিয়েছি। তবে এগুলো আরও অনেক কাজ পরিচালনা করতে পারে, যেমন টেক্সট জেনারেশন এবং মেশিন অনুবাদ — আমরা পরবর্তী ইউনিটে এই কাজগুলো নিয়ে আলোচনা করব।



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিকতার জন্য চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
